In [ ]:
%matplotlib inline
import os, sys, re
import glob

import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn

from random import randrange
from PIL import Image
import matplotlib.pyplot as plt

!pip install opencv-python -qqq
!pip install wandb -qqq
import wandb
wandb.login()

In [ ]:
import argparse
""" Training and hyperparameter search configurations """
curr_dir = os.getcwd()

parser = argparse.ArgumentParser(description='Final')
parser.add_argument('--img_dir', type=str, default='/Users/valenetjong/alzheimer-classification/data',
                    help='directory for image storage')
parser.add_argument('--seed', type=int, default=1,
                    help='random seed (default: 1)')
parser.add_argument('--num_classes', type=int, default=3,
                    help='number of classes')
parser.add_argument('--loss', type=str, default="cross entropy",
                    help='cross entropy, focal')
parser.add_argument('--download_oasis', type=bool, default=False,
                    help="download oasis dataset from links if True, use already extracted files, if False")
parser.add_argument('--process_flag', type=bool, default=False,
                    help="extract files from disk if True, use already extracted files, if False")
parser.add_argument('--create_dataset', type=bool, default=False,
                    help="create dataset from scratch if True, load in processed dataset if False")
parser.add_argument('--transforms', type=str, default='all',
                    help='transforms for data augmentation')
parser.add_argument('--threshold', type=float, default=3e-4,
                    help='early stopping criterion')
args = parser.parse_args('')
# Set random seed to reproduce results
torch.manual_seed(args.seed)

In [ ]:
""" Set-up wandb """
sweep_config = {
    'method': 'bayes'
    }

metric = {
    'name': 'max val acc',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

params = {
    'max_epochs': {
        'value': 250
        },
    'hidden_size': {
        'values': [8, 16],
        },
    'fc_size': {
        'values': [32, 64, 128, 256, 512]
        },
    'conv_in_size': {
        'values': [32, 64, 128, 256]
        },
    'conv_hid_size': {
        'values': [8, 16, 32]
        },
    'conv_out_size': {
        'values': [8, 16, 32]
        },
    'dropout': {
          'values': [0.15, 0.2, 0.25, 0.3]
        },
    'batch_size': {
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 8,
        'max': 64,
        },
    'lr': {
        'values': [1e-3, 1e-4, 1e-5]
        },
    }

sweep_config['parameters'] = params
sweep_id = wandb.sweep(sweep_config, project="2D-masked-imgs")

### Download Files
Available at: https://www.kaggle.com/datasets/ninadaithal/imagesoasis/download?datasetVersionNumber=1